    16_230109_100_kyungki_apt_성남
   
     data : /aiffel/aiffel/000_aiffelthon_230111/3김연수/data_dir/apt_성남_402.xls

In [1]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [2]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [3]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [5]:
#9 Folium를 활용한 공간 데이터 시각화
### 전국도시철도역사정보표준데이터 가져오기

apt_df = pd.read_excel("/aiffel/aiffel/000_aiffelthon_230111/3김연수/data_dir/apt_성남_402.xls")
apt_df.head()

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

In [7]:
apt_df.drop(["공동주택구분"], axis=1,inplace=True)
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부
0,성남시,LH위례31단지,창곡동,500,2017-07-07,15,13,2334,N
1,성남시,LH위례36단지,창곡동,501,2021-01-04,15,3,470,N
2,성남시,MD아파트,수진동,2220,2001-12-10,11,1,63,N
3,성남시,SR HIGHONE 하이원,성남동,3755,2004-05-21,12,1,44,N
4,성남시,e편한세상 테라스위례,창곡동,575,2017-11-28,4,15,360,N


In [8]:
apt_df['지번주소'] = apt_df['지번주소'].astype(str)

In [9]:
#okok
apt_df['addr'] = '경기도 ' + apt_df[['시군명', '읍면동주소','지번주소']].agg(' '.join, axis=1)
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr
0,성남시,LH위례31단지,창곡동,500,2017-07-07,15,13,2334,N,경기도 성남시 창곡동 500
1,성남시,LH위례36단지,창곡동,501,2021-01-04,15,3,470,N,경기도 성남시 창곡동 501
2,성남시,MD아파트,수진동,2220,2001-12-10,11,1,63,N,경기도 성남시 수진동 2220
3,성남시,SR HIGHONE 하이원,성남동,3755,2004-05-21,12,1,44,N,경기도 성남시 성남동 3755
4,성남시,e편한세상 테라스위례,창곡동,575,2017-11-28,4,15,360,N,경기도 성남시 창곡동 575


In [10]:
# https://sgisapi.kostat.go.kr/OpenAPI3/addr/stage.json

In [11]:
#5 seoul university   # 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)

    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

슝~


In [14]:
#5 seoul university .. 주소 기반 위도,경도 추가
cnt = 0
for _, row in apt_df.iterrows():
    try:
        x, y = get_adress_x_y(row["addr"])
     
        apt_df.loc[_, "x"] = x
        apt_df.loc[_, "y"] = y
    except:
        print(row["addr"] + '은(는) 검색에 실패했습니다')
        cnt = cnt + 1

print("실패건수 : ", cnt)
apt_df.head()


경기도 성남시 태평동 18264은(는) 검색에 실패했습니다
경기도 성남시 금곡동 181은(는) 검색에 실패했습니다
경기도 성남시 신흥동 1330274은(는) 검색에 실패했습니다
경기도 성남시 신흥동 6951은(는) 검색에 실패했습니다
경기도 성남시 창곡동 3505은(는) 검색에 실패했습니다
경기도 성남시 상대원동 44670은(는) 검색에 실패했습니다
경기도 성남시 신흥동 205634은(는) 검색에 실패했습니다
경기도 성남시 정자동 44897은(는) 검색에 실패했습니다
경기도 성남시 신흥동 206546은(는) 검색에 실패했습니다
경기도 성남시 신흥동 205999은(는) 검색에 실패했습니다
경기도 성남시 정자동 44835은(는) 검색에 실패했습니다
경기도 성남시 창곡동 44602은(는) 검색에 실패했습니다
경기도 성남시 은행동 11841은(는) 검색에 실패했습니다
경기도 성남시 태평동 1173707은(는) 검색에 실패했습니다
경기도 성남시 정자동 44589은(는) 검색에 실패했습니다
경기도 성남시 정자동 44666은(는) 검색에 실패했습니다
경기도 성남시 중앙동 44717은(는) 검색에 실패했습니다
경기도 성남시 태평동 1968234은(는) 검색에 실패했습니다
경기도 성남시 시흥동 320은(는) 검색에 실패했습니다
경기도 성남시 태평동 398513은(는) 검색에 실패했습니다
경기도 성남시 시흥동 19360은(는) 검색에 실패했습니다
경기도 성남시 구미동 82, 83, 84, 85은(는) 검색에 실패했습니다
경기도 성남시 상대원동 399272은(는) 검색에 실패했습니다
경기도 성남시 금광동 200886은(는) 검색에 실패했습니다
실패건수 :  24


,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y
0,성남시,LH위례31단지,창곡동,500,2017-07-07,15,13,2334,N,경기도 성남시 창곡동 500,967703.547137337,1941582.07855826
1,성남시,LH위례36단지,창곡동,501,2021-01-04,15,3,470,N,경기도 성남시 창곡동 501,967698.786889659,1941481.19446715
2,성남시,MD아파트,수진동,2220,2001-12-10,11,1,63,N,경기도 성남시 수진동 2220,968039.240912453,1937592.44979592
3,성남시,SR HIGHONE 하이원,성남동,3755,2004-05-21,12,1,44,N,경기도 성남시 성남동 3755,967748.617029319,1937060.16420183
4,성남시,e편한세상 테라스위례,창곡동,575,2017-11-28,4,15,360,N,경기도 성남시 창곡동 575,969222.640576669,1940851.71070132


In [15]:
#5 seoul university  ..# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')


슝~


In [16]:
#5 seoul university  .. 좌표계가 UTM-K이므로 WSG84좌표계로 변환
for _, row in apt_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        apt_df.loc[_, "lng"] = x
        apt_df.loc[_, "lat"] = y

    except:
        pass
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y,lng,lat
0,성남시,LH위례31단지,창곡동,500,2017-07-07,15,13,2334,N,경기도 성남시 창곡동 500,967703.547137337,1941582.07855826,127.134747,37.472897
1,성남시,LH위례36단지,창곡동,501,2021-01-04,15,3,470,N,경기도 성남시 창곡동 501,967698.786889659,1941481.19446715,127.134698,37.471988
2,성남시,MD아파트,수진동,2220,2001-12-10,11,1,63,N,경기도 성남시 수진동 2220,968039.240912453,1937592.44979592,127.138717,37.436948
3,성남시,SR HIGHONE 하이원,성남동,3755,2004-05-21,12,1,44,N,경기도 성남시 성남동 3755,967748.617029319,1937060.16420183,127.135455,37.432140
4,성남시,e편한세상 테라스위례,창곡동,575,2017-11-28,4,15,360,N,경기도 성남시 창곡동 575,969222.640576669,1940851.71070132,127.151957,37.466366


In [17]:
#5 seoul university  .. 시각화 .. 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

슝~


In [ ]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in apt_df.iterrows():
    try:
        folium.Marker(
            location=[apt_df.loc[idx, "lat"], apt_df.loc[idx, "lng"]],
            popup=apt_df.loc[idx, "공동주택명정보"],
            icon=folium.Icon(color="cadetblue", icon="house")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

In [22]:
#5 seoul university 
map

In [23]:
map.save("/aiffel/aiffel/000_aiffelthon_230111/3김연수/003_map_xml/apt_sungnam.html")

In [ ]:
apt_df.to_csv("apt_sungnam.csv", index = False)